In [2]:
# Graph 1

import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

def get_mongo_data(database_name, collection_name):
    client = MongoClient('mongodb://localhost:27017/')  # Connect to MongoDB
    db = client[database_name]  # Select the database
    collection = db[collection_name]  # Select the collection
    return pd.DataFrame(list(collection.find()))  # Retrieve data from the collection and convert to DataFrame

def plot_fossil_energy_consumption():
    country = input("Enter the country name: ").lower()

    # Load data from MongoDB
    energy_data = get_mongo_data('OurWorldInData', 'owid-energy-data')

    # Filter data for the specified country
    country_data = energy_data[energy_data['country'].str.lower() == country]

    # Plot fossil energy consumption
    plt.plot(country_data['year'], country_data['coal_consumption'], label='Coal Consumption')
    plt.plot(country_data['year'], country_data['oil_consumption'], label='Oil Consumption')
    plt.plot(country_data['year'], country_data['gas_consumption'], label='Gas Consumption')

    plt.xlabel('Year')
    plt.ylabel('Energy Consumption (TWh)')
    plt.title(f'Fossil Energy Consumption in {country.capitalize()}')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    plot_fossil_energy_consumption()


KeyError: 'country'

In [1]:
# Graph 2

import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

def get_mongo_data(database_name, collection_name):
    client = MongoClient('mongodb://localhost:27017/')  # Connect to MongoDB
    db = client[database_name]  # Select the database
    collection = db[collection_name]  # Select the collection
    return pd.DataFrame(list(collection.find()))  # Retrieve data from the collection and convert to DataFrame

def plot_sustainable_energy_pie():
    year = int(input("Enter the year: "))
    countries = input("Enter up to 4 country names separated by comma: ").lower().split(',')

    # Load data from MongoDB
    energy_data = get_mongo_data('OurWorldInData', 'owid-energy-data')

    # Filter data for the specified year and countries
    year_data = energy_data[energy_data['year'] == year]
    country_data = year_data[year_data['country'].str.lower().isin(countries)]

    # Plot pie charts for each country
    num_countries = len(countries)
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle(f'Sustainable Energy Consumption in {", ".join(countries)} ({year})')

    colors = ['#512da8', '#0085b0', '#00c3b2', '#acfa70']  # Hexadecimal color codes for Hydro, Solar, Biofuel, and Wind respectively

    for i, country in enumerate(countries):
        row_index = i // 2
        col_index = i % 2

        country_row = country_data[country_data['country'].str.lower() == country.lower()]
        if not country_row.empty:
            axes[row_index, col_index].pie(country_row[['hydro_consumption', 'solar_consumption', 'biofuel_consumption', 'wind_consumption']].values[0],
                                           labels=['', '', '', ''], autopct='%1.1f%%', colors=colors)
            axes[row_index, col_index].set_title(country_row['country'].values[0].capitalize())
        else:
            axes[row_index, col_index].axis('off')
            axes[row_index, col_index].text(0.5, 0.5, f'No data available for {country}', horizontalalignment='center', verticalalignment='center', fontsize=12)

    if num_countries < 4:
        for i in range(num_countries, 4):
            row_index = i // 2
            col_index = i % 2
            fig.delaxes(axes[row_index, col_index])

    fig.legend(['Hydro', 'Solar', 'Biofuel', 'Wind'], loc='upper right')
    plt.show()

if __name__ == "__main__":
    plot_sustainable_energy_pie()


KeyError: 'year'

In [ ]:
# Graph 3

import pymongo
import matplotlib.pyplot as plt
import numpy as np

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["OurWorldInData"]

def generate_graph(data, argument, years):
    # Group data by country
    country_data = {}
    for entry in data:
        country = entry['country']
        year = entry['year']
        value = entry.get(argument, 0)
        if country not in country_data:
            country_data[country] = {year: [value]}  # Store value in a list
        else:
            if year not in country_data[country]:
                country_data[country][year] = [value]
            else:
                country_data[country][year].append(value)  # Append value to existing list
    
    # Calculate mean value for each country
    country_means = {country: {year: np.mean(values) for year, values in data.items()} for country, data in country_data.items()}

    top_countries = []
    count = 0
    for country, year_means in sorted(country_means.items(), key=lambda x: np.mean(list(x[1].values())), reverse=True):
        print("Country:", country)
        print("Year means:", year_means)
        mean_value = np.mean(list(year_means.values()))
        print("Mean value:", mean_value)
        if db["countries"].count_documents({"country": country}) > 0 and country != "World":
            top_countries.append((country, mean_value))
            count += 1
            if count == 5:
                break

    print("Top countries after filtering:", top_countries)
    
    # Add Canada to the top countries
    top_countries.append(("Canada", 0))  # Add Canada with a mean value of 0 for now
    
    # Specify colors for each country
    colors = ['#512da8', '#006daf', '#0090b4', '#00b5b7', '#00e195', '#acfa70'][:len(top_countries)]
    
    # Plot the data
    num_years = len(years)
    bar_width = 0.15
    index = np.arange(num_years)
    
    for i, (country, _) in enumerate(top_countries):
        country_values = [country_data.get(country, {}).get(year, [0])[0] for year in years]
        plt.bar(index + i * bar_width, country_values, bar_width, label=country, color=colors[i])
    
    plt.xlabel('Year')
    plt.ylabel(argument.capitalize())
    plt.title(f'Top 5 Countries and Canada: {argument.capitalize()} ({min(years)}-{max(years)})')
    plt.xticks(index + 0.3, years)
    plt.legend()
    plt.grid(True)  # Add grid
    plt.tight_layout()
    plt.show()

# Prompt user to choose the argument
print("Choose the category:\n1. Greenhouse gas emissions\n2. Population\n3. GDP")
choice = input("Enter your choice (1/2/3): ")

# Map user choice to the corresponding argument
arguments = {
    "1": "greenhouse_gas_emissions",
    "2": "population",
    "3": "gdp"
}

# Verify user input
if choice in arguments:
    argument = arguments[choice]
else:
    print("Invalid choice. Please enter a valid option.")
    exit()

# Prompt user for the ending year
end_year = int(input("Enter the ending year: "))

# Specify the years
years = list(range(end_year - 9, end_year + 1))

# Specify the collection name
collection_name = 'owid-energy-data'

# Query the MongoDB collection
collection = db[collection_name]
data = list(collection.find({'year': {'$in': years}}))

# Generate the graph
generate_graph(data, argument, years)
